In [2]:
import numpy as np
import pandas as pd

In [ ]:
def q10(x):
    return x.quantile(0.1)

def q90(x):
    return x.quantile(0.9)

def q30(x):
    return x.quantile(0.3)

def q70(x):
    return x.quantile(0.7)

In [1]:
import numpy as np
import pandas as pd

In [2]:
from tsfresh.feature_extraction.feature_calculators import longest_strike_above_mean, mean_abs_change, \
    mean_second_derivative_central, sample_entropy, benford_correlation, count_above_mean,\
    percentage_of_reoccurring_datapoints_to_all_datapoints, variation_coefficient

In [3]:
account_static = pd.read_csv('./data/账户静态信息.csv')
account_trade = pd.read_csv('./data/账户交易信息.csv')
y_train = pd.read_csv('./data/训练集标签.csv')
y_test = pd.read_csv('./data/test_dataset.csv')

In [4]:
account_trade['dfhh'].unique().shape

(2816,)

In [5]:
account_trade['jyqd'].unique().shape

(21,)

In [6]:
account_trade['zydh'].unique().shape

(158,)

In [13]:
account_trade['jyje_label'] = pd.qcut(account_trade['jyje'], 10, labels=range(10))

In [14]:
for col in ['jyje_label']:
    tmp = account_trade.merge(y_train, how='left', on='zhdh').groupby(['black_flag', col])\
        .size().reset_index().rename({0: f'flag_{col}_cnt'},axis=1)
    tmp2 = account_trade.merge(y_train, how='left', on='zhdh').groupby('black_flag').size()\
        .reset_index().rename({0:'flag_cnt'},axis=1)
    tmp = tmp.merge(tmp2, how='left', on='black_flag')
    tmp[f'p({col}|flag)'] = tmp[f'flag_{col}_cnt'] / tmp['flag_cnt']
    for i in range(2):
        tmp3 = account_trade.merge(tmp[tmp['black_flag'] == i][[col, f'p({col}|flag)']], how='left', on=col)\
            .groupby('zhdh').agg({f'p({col}|flag)': ['mean', 'sum', 'std', 'max']})
        tmp3.columns = [f'{c[0]}_{c[1]}_lag={i}' for c in tmp3.columns]
        tmp3 = tmp3.reset_index()


In [4]:
account_trade.columns

Index(['jylsxh', 'zhdh', 'dfzh', 'jdbj', 'jyje', 'zhye', 'dfhh', 'jyrq',
       'jysj', 'jyqd', 'zydh', 'dfmccd'],
      dtype='object')

In [38]:
col = 'jyje'
tmp = account_trade.groupby(['zhdh', 'jyrq', 'jdbj'])[col].sum().reset_index()
tmp = tmp.pivot(index=['zhdh', 'jyrq'], columns=['jdbj'], values=[col])
tmp.columns = [f'jdbj_0_{col}', f'jdbj_1_{col}']

tmp = tmp.reset_index().fillna(0)
tmp[col] = tmp[f'jdbj_0_{col}'] + tmp[f'jdbj_1_{col}']
tmp[f'jdbj_0_{col}_ratio'] = tmp[f'jdbj_0_{col}'] / tmp[col]
tmp[f'jdbj_1_{col}_ratio'] = tmp[f'jdbj_1_{col}'] / tmp[col]
tmp.groupby('zhdh')[[f'jdbj_0_{col}_ratio', f'jdbj_1_{col}_ratio']].sum()

,jdbj_0_jyje_ratio,jdbj_1_jyje_ratio
zhdh,,
00037295453A928A,26.999945,22.000055
0004CC075464D54B,7.667221,4.332779
000AA77144DC1BCC,56.444106,27.555894
001ABBF3373AFC5B,11.835155,12.164845
00310769938BC172,12.288785,19.711215
...,...,...
FFC065BECA5E133C,16.168266,70.831734
FFD2E9065F19B38D,47.901217,8.098783
FFD48BA98FA2D299,40.690941,4.309059


In [34]:
col1 = 'jyqd'
col2 = 'jyrq'

tmp = account_trade.groupby(['zhdh', col1])[col2].size().reset_index().rename(columns={col2: f'{col1}_{col2}_cnt'})
tmp2 = account_trade.groupby(['zhdh'])[col2].size().reset_index().rename(columns={col2: f'{col2}_cnt'})
tmp = tmp.merge(tmp2, how='left', on='zhdh')
tmp[f'{col1}_{col2}_ratio'] = tmp[f'{col1}_{col2}_cnt'] / tmp[f'{col2}_cnt']
tmp = tmp.groupby('zhdh')[f'{col1}_{col2}_ratio'].agg(['mean', 'max', 'std'])
tmp.columns = [f'{col1}_{col2}_ratio_mean', f'{col1}_{col2}_ratio_max', f'{col1}_{col2}_ratio_std']
tmp = tmp.reset_index()

In [39]:
from collections import Counter

In [40]:
def calc_cf(values):
    rms = np.sqrt(np.mean(values ** 2))
    xr = np.mean(np.abs(values)) ** 2
    max_v = values.max()
    min_v = values.min()
    pk_max = max(max_v, -min_v)
    return pk_max / rms if rms != 0 else 0

def calc_pf(values):
    rms = np.sqrt(np.mean(values ** 2))
    xr = np.mean(np.abs(values)) ** 2
    max_v = values.max()
    min_v = values.min()
    pk_max = max(max_v, -min_v)
    return pk_max / xr if xr != 0 else 0

def top1_num(values):
    counter = Counter(values)
    return counter.most_common(3)[0][0]

def top1_ratio(values):
    counter = Counter(values)
    return counter.most_common(3)[0][1] / len(values)

def top2_num(values):
    counter = Counter(values)
    return counter.most_common(3)[1][0] if len(counter) > 1 else -1

def top2_ratio(values):
    counter = Counter(values)
    return counter.most_common(3)[1][1] / len(values) if len(counter) > 1 else 0

def top3_num(values):
    counter = Counter(values)
    return counter.most_common(3)[2][0] if len(counter) > 2 else -1

def top3_ratio(values):
    counter = Counter(values)
    return counter.most_common(3)[2][1] / len(values) if len(counter) > 2 else 0

In [43]:
tmp = account_trade.groupby(['zhdh', 'jdbj'])['jyje']\
    .agg([calc_cf, calc_pf, top1_num, top1_ratio,
          top2_num, top2_ratio, top3_num, top3_ratio]).reset_index()

In [49]:
tmp = tmp.pivot(index='zhdh', columns=['jdbj'],
          values=['calc_cf', 'calc_pf', 'top1_num', 'top1_ratio',
                  'top2_num', 'top2_ratio', 'top3_num', 'top3_ratio'])

In [52]:
tmp.columns = [f'{c[0]}_jdbj{c[1]}' for c in tmp.columns]
tmp = tmp.reset_index()

In [53]:
tmp

,zhdh,calc_cf_jdbj0,calc_cf_jdbj1,calc_pf_jdbj0,calc_pf_jdbj1,top1_num_jdbj0,top1_num_jdbj1,top1_ratio_jdbj0,top1_ratio_jdbj1,top2_num_jdbj0,top2_num_jdbj1,top2_ratio_jdbj0,top2_ratio_jdbj1,top3_num_jdbj0,top3_num_jdbj1,top3_ratio_jdbj0,top3_ratio_jdbj1
0,00037295453A928A,5.124747,4.826685,0.002412,0.001737,1200.0,3000.00,0.037037,0.059701,1.0,5000.0,0.037037,0.029851,2702.39,4500.0,0.037037,0.029851
1,0004CC075464D54B,3.485060,2.455481,0.001269,0.000217,1.0,20000.00,0.346154,0.181818,22120.0,5000.0,0.038462,0.181818,25000.00,25000.0,0.038462,0.090909
2,000AA77144DC1BCC,15.231103,4.963733,0.413435,0.006866,0.1,1000.00,0.295950,0.208333,3.0,500.0,0.112150,0.197917,2.00,2000.0,0.037383,0.093750
3,001ABBF3373AFC5B,1.850582,2.539697,0.002177,0.003067,1000.0,549.00,0.200000,0.062500,1.0,550.0,0.200000,0.062500,3000.00,1069.0,0.200000,0.062500
4,00310769938BC172,1.619629,1.642098,0.000020,0.000021,10000.0,60000.00,0.250000,0.086957,200000.0,139960.0,0.250000,0.043478,56830.00,127230.0,0.062500,0.043478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,FFC065BECA5E133C,2.170690,4.725396,0.000193,0.003130,50000.0,358.00,0.136364,0.018382,2653.0,448.0,0.068182,0.018382,1.00,438.0,0.068182,0.018382
5996,FFD2E9065F19B38D,3.821146,1.816664,0.021988,0.001123,500.0,2439.06,0.068966,0.153846,13.0,500.0,0.057471,0.153846,5.00,3800.0,0.057471,0.076923
5997,FFD48BA98FA2D299,5.731143,1.388252,0.088323,0.000497,0.8,4000.00,0.259259,0.285714,2.4,5000.0,0.222222,0.285714,1.60,2098.0,0.061728,0.285714
5998,FFD8F2A4DB42AC6D,2.312991,6.031459,0.000308,0.025684,10000.0,100.00,0.078125,0.026891,16000.0,102.0,0.046875,0.023529,15000.00,101.0,0.046875,0.015126


In [41]:
account_trade.groupby('zhdh')['jyje'].agg([calc_cf, calc_pf, top1_num, top1_ratio,
                                           top2_num, top2_ratio, top3_num, top3_ratio])

,calc_cf,calc_pf,top1_num,top1_ratio,top2_num,top2_ratio,top3_num,top3_ratio
zhdh,,,,,,,,
00037295453A928A,4.983150,0.002065,1200.0,0.027027,3000.0,0.027027,5000.0,0.027027
0004CC075464D54B,3.439945,0.000707,1.0,0.243243,20000.0,0.081081,5000.0,0.081081
000AA77144DC1BCC,12.253670,0.136612,0.1,0.257453,3.0,0.097561,500.0,0.035230
001ABBF3373AFC5B,2.335006,0.002796,1000.0,0.096774,1.0,0.096774,3000.0,0.096774
00310769938BC172,1.889012,0.000024,10000.0,0.102564,200000.0,0.102564,60000.0,0.051282
...,...,...,...,...,...,...,...,...
FFC065BECA5E133C,5.100953,0.002336,50000.0,0.018987,358.0,0.015823,448.0,0.015823
FFD2E9065F19B38D,4.240552,0.015179,500.0,0.080000,13.0,0.060000,5.0,0.050000
FFD48BA98FA2D299,4.202247,0.024785,0.8,0.238636,2.4,0.204545,1.6,0.056818


In [6]:
tmp = account_trade.sort_values(by='zhye').groupby('zhdh').apply(lambda x: x.tail(1)['jyqd'].values[0])\
    .reset_index().rename(columns={0: 'min_zhye_jyqd'})

,zhdh,min_zhye_jyqd
0,00037295453A928A,E96ED478
1,0004CC075464D54B,621461AF
2,000AA77144DC1BCC,E96ED478
3,001ABBF3373AFC5B,621461AF
4,00310769938BC172,621461AF
...,...,...
5995,FFC065BECA5E133C,274AD478
5996,FFD2E9065F19B38D,854D6FAE
5997,FFD48BA98FA2D299,621461AF
5998,FFD8F2A4DB42AC6D,621461AF


In [7]:
account_trade.sort_values(by='zhye').groupby('zhdh').apply(lambda x: x.tail(1)['zydh'].values[0])

zhdh
00037295453A928A    C22ABFA3
0004CC075464D54B    A3C65C29
000AA77144DC1BCC    C22ABFA3
001ABBF3373AFC5B    A3C65C29
00310769938BC172    A3C65C29
                      ...   
FFC065BECA5E133C    2618045A
FFD2E9065F19B38D    DC5C7986
FFD48BA98FA2D299    A3C65C29
FFD8F2A4DB42AC6D    A3C65C29
FFEC4454D961DEB1    A3C65C29
Length: 6000, dtype: object

In [21]:
account_trade['jyje_new'] = account_trade['jyje'] * account_trade['jdbj'].apply(lambda x: -1 if x == 0 else 1)

In [23]:
tmp = account_trade.groupby('zhdh').agg({
    'jyje_new': ['sum','mean','median','max','min','std',np.ptp,
                longest_strike_above_mean, mean_abs_change, mean_second_derivative_central,
                 sample_entropy, benford_correlation, count_above_mean,
                 percentage_of_reoccurring_datapoints_to_all_datapoints]
})

/Users/yitinglin/Projects/dcic2023/env/lib/python3.7/site-packages/tsfresh/feature_extraction/feature_calculators.py:1739: RuntimeWarning: divide by zero encountered in log
  return -np.log(A / B)
/Users/yitinglin/Projects/dcic2023/env/lib/python3.7/site-packages/tsfresh/feature_extraction/feature_calculators.py:1739: RuntimeWarning: invalid value encountered in long_scalars
  return -np.log(A / B)


In [26]:
tmp.columns = [f'{c[0]}_{c[1]}' for c in tmp.columns]

In [28]:
tmp = tmp.reset_index()

In [29]:
tmp

,zhdh,jyje_new_sum,jyje_new_mean,jyje_new_median,jyje_new_max,jyje_new_min,jyje_new_std,jyje_new_ptp,jyje_new_longest_strike_above_mean,jyje_new_mean_abs_change,jyje_new_mean_second_derivative_central,jyje_new_sample_entropy,jyje_new_benford_correlation,jyje_new_count_above_mean,jyje_new_percentage_of_reoccurring_datapoints_to_all_datapoints
0,00037295453A928A,2612.60,17.652703,-34.800,24850.0,-24850.00,5003.707328,49700.00,3,6324.360408,10.784110,1.559958,0.851240,68,0.141892
1,0004CC075464D54B,22885.50,618.527027,-1.000,50000.0,-39000.00,14722.262843,89000.00,2,15934.194444,601.714286,1.033015,0.885034,11,0.351351
2,000AA77144DC1BCC,-20.03,-0.027141,-3.000,10000.0,-13469.46,1099.963953,23469.46,3,551.315577,0.679348,0.282069,0.977746,98,0.788618
3,001ABBF3373AFC5B,-502.00,-16.193548,355.000,3507.0,-3000.00,1526.661721,6507.00,4,1959.916667,17.163793,2.484907,0.735303,21,0.354839
4,00310769938BC172,323662.50,8299.038462,60000.000,150500.0,-200000.00,106929.498428,350500.00,7,129308.684211,-1365.878378,1.734601,0.614446,22,0.256410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,FFC065BECA5E133C,44441.93,140.639019,512.000,24000.0,-50000.00,9816.625120,74000.00,42,6710.277651,1.167038,0.619885,0.934672,271,0.297468
5996,FFD2E9065F19B38D,-2436.64,-24.366400,-22.500,4600.0,-2400.00,1089.954093,7000.00,4,934.580202,1.392857,0.839142,0.905528,50,0.680000
5997,FFD48BA98FA2D299,1074.57,12.211023,-2.400,5000.0,-6006.00,1437.373359,11006.00,1,969.803908,-0.340116,0.296577,0.434956,6,0.590909
5998,FFD8F2A4DB42AC6D,968.91,0.772656,199.865,10800.0,-22382.00,2331.451525,33182.00,68,1386.206313,0.007983,0.484256,0.956672,1191,0.562201


In [17]:
from tsfresh.feature_extraction.feature_calculators import variation_coefficient

In [22]:
account_trade[account_trade['jdbj'] == 1].groupby('zhdh')['jyje'].apply(lambda x: variation_coefficient(x))\
    .reset_index().rename(columns={'jyje': 'jyje_var_coeff'})

,zhdh,jyje_var_coeff
0,00037295453A928A,0.923551
1,0004CC075464D54B,0.893599
2,000AA77144DC1BCC,1.336717
3,001ABBF3373AFC5B,0.817203
4,00310769938BC172,0.428530
...,...,...
5993,FFC065BECA5E133C,1.537596
5994,FFD2E9065F19B38D,0.752181
5995,FFD48BA98FA2D299,0.538595
5996,FFD8F2A4DB42AC6D,1.590816


In [23]:
account_trade[account_trade['jdbj'] == 0].groupby('zhdh')['jyje'].apply(lambda x: variation_coefficient(x))\
    .reset_index().rename(columns={'jyje': 'jyje_var_coeff'})

,zhdh,jyje_var_coeff
0,00037295453A928A,1.205264
1,0004CC075464D54B,1.753546
2,000AA77144DC1BCC,4.796571
3,001ABBF3373AFC5B,0.952224
4,00310769938BC172,0.706189
...,...,...
5992,FFC065BECA5E133C,1.022207
5993,FFD2E9065F19B38D,1.616837
5994,FFD48BA98FA2D299,3.892328
5995,FFD8F2A4DB42AC6D,0.629391


In [4]:
account_trade.columns

Index(['jylsxh', 'zhdh', 'dfzh', 'jdbj', 'jyje', 'zhye', 'dfhh', 'jyrq',
       'jysj', 'jyqd', 'zydh', 'dfmccd'],
      dtype='object')

In [6]:
account_trade['jyje']

0          310.00
1          599.99
2         4000.00
3          299.98
4          999.96
           ...   
816265    1000.00
816266    5000.00
816267      18.90
816268     681.70
816269       1.00
Name: jyje, Length: 816270, dtype: float64

In [11]:
account_trade['jyje_10_times'] = (account_trade['jyje'] % 10 == 0) + 0
account_trade['jyje_100_times'] = (account_trade['jyje'] % 100 == 0) + 0

In [12]:
account_trade.groupby('zhdh')['jyje_100_times'].sum()

zhdh
00037295453A928A     33
0004CC075464D54B     15
000AA77144DC1BCC    123
001ABBF3373AFC5B     10
00310769938BC172     18
                   ... 
FFC065BECA5E133C     26
FFD2E9065F19B38D     29
FFD48BA98FA2D299      6
FFD8F2A4DB42AC6D    138
FFEC4454D961DEB1     35
Name: jyje_100_times, Length: 6000, dtype: int64

In [23]:
account_trade.columns

Index(['jylsxh', 'zhdh', 'dfzh', 'jdbj', 'jyje', 'zhye', 'dfhh', 'jyrq',
       'jysj', 'jyqd', 'zydh', 'dfmccd'],
      dtype='object')

In [31]:
account_trade[account_trade['jdbj'] == 0].groupby(['zhdh', 'jyqd']).size().reset_index().groupby('zhdh')\
        .apply(lambda x: x.sort_values(by=0).iloc[-1]['jyqd'])\
        .reset_index().rename(columns={0: f'most_jyqd'})

,zhdh,most_jyqd
0,00037295453A928A,E96ED478
1,0004CC075464D54B,E96ED478
2,000AA77144DC1BCC,E96ED478
3,001ABBF3373AFC5B,757B505C
4,00310769938BC172,091D584F
...,...,...
5992,FFC065BECA5E133C,091D584F
5993,FFD2E9065F19B38D,E96ED478
5994,FFD48BA98FA2D299,757B505C
5995,FFD8F2A4DB42AC6D,091D584F


In [32]:
account_trade[account_trade['jdbj'] == 1].groupby(['zhdh', 'jyqd']).size().reset_index().groupby('zhdh')\
        .apply(lambda x: x.sort_values(by=0).iloc[-1]['jyqd'])\
        .reset_index().rename(columns={0: f'most_jyqd'})

,zhdh,most_jyqd
0,00037295453A928A,757B505C
1,0004CC075464D54B,621461AF
2,000AA77144DC1BCC,E96ED478
3,001ABBF3373AFC5B,621461AF
4,00310769938BC172,621461AF
...,...,...
5993,FFC065BECA5E133C,274AD478
5994,FFD2E9065F19B38D,E96ED478
5995,FFD48BA98FA2D299,621461AF
5996,FFD8F2A4DB42AC6D,621461AF
